# 目的

#每日收盤後開始爬股價

#可以視情況條種是否抓營收，大概都隔月的10 後營收才會公布

## 【資料源】
來自 _BaseInfo  _CrawBase 會先去取基本相關資料 (股票號碼...等)

## 【輸出】
股票相關推薦圖表/數據

In [6]:
import pandas as pd

#  Update Base Info

In [7]:
 
#%run _BaseInfoETF.ipynb
%run _CrawBase.ipynb
#%run _LineMsg.ipynb
%run _genHtml.ipynb

37.7
log/RunRealTimeStock/2024-11-08DailyResultWeb.xlsx


%run RunRealTimeStock.ipynb

In [8]:
def MainStock(stock_number, start_month,end_time=datetime.now().strftime("%Y-%m-%d"),updateRevenue=False):  
    strmsg ='【stock_number】'+stock_number+'\n' +'【start_month】'+start_month+'\n'+'【end_time】'+end_time+'\n' 
    try:
        #處理 Stock
        RowData_df_craw_stock,His_Stock ,isSuccess= craw_stock(stock_number,start_month,end_time)
        #data_final,df_final=DataCleaning(RowData_df_craw_stock)
        print("【craw_stock】 success")
        
        #處理 Revenue
        if(updateRevenue):
            RowData_df_craw_revenue,His_Stock=craw_revenue(stock_number)
            df_Revenue=RevenueDataCleaning(RowData_df_craw_revenue)
            print("【craw_revenue】 success")
            
        #處理相關指標轉成html 
        stock_data=data_process(RowData_df_craw_stock)
        
        level,interval_type, lower_bound, upper_bound=get_interval(stock_data.iloc[-1]['收盤價'],np.array(stock_data['收盤價']))
        stock_data['stock_number']=stock_number
        stock_data['level']=level
        stock_data['interval_type']=interval_type
        stock_data['lower_bound']=lower_bound
        stock_data['upper_bound']=upper_bound
        save_df_to_excel(stock_data)
        gen_html(stock_number,stock_data)

        #sendLineMessage('成功'+'\n'+strmsg )
    except:
        #sendLineMessage('失败'+'\n'+strmsg)
        print('MainStock Fail'+'\n'+strmsg)
        
    #return data_final,df_final,df_Revenue
    

# 清資料使用
type_='Stock_His'
strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')
df=get_DB()
df[~((df['status'] != 'OK')& (df['status'] != '很抱歉，沒有符合條件的資料!'))].to_csv('BaseInfo/His_Stock.csv',index=False) 
DDD=df[((df['status'] != 'OK')& (df['status'] != '很抱歉，沒有符合條件的資料!'))]['stock_number'].unique()

start_month='2023-01-01'  #起始月份
want_update_Stock=True
want_update_Revenue=False
file_path = 'D:\Project\Jupyter\Stock\Main\DailyResultLast.xlsx'   

stock_number=str(6763)
if (want_update_Stock):        
    type_='Stock_His'
    strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')
    delete_DB_stock_by_Month_Stock_number(strMonth,stock_number,type_)
try:
    if(want_update_Revenue):
        MainStock(stock_number,start_month,want_update_Revenue)
    else:
        MainStock(stock_number,start_month)
except Exception as error:
    print("An exception occurred:", error)
    print('Action Fail'+'\n')

,公司代號,公司名稱,公司簡稱,產業別,實收資本額,Type,上市公司產業類別,上櫃公司產業類別,新編碼
0,1240,茂生農經股份有限公司,茂生農經,33,413386660,上櫃,--,農業科技業,33
1,1259,安心食品服務股份有限公司,安心,16,323895000,上櫃,觀光事業,觀光事業,16
2,1264,德麥食品股份有限公司,德麥,02,370643670,上櫃,食品工業,食品工業,02
3,1268,漢來美食股份有限公司,漢來美食,16,422237300,上櫃,觀光事業,觀光事業,16
4,1294,漢田生技股份有限公司,漢田生技,02,276007630,上櫃,食品工業,食品工業,02
5,1336,台翰精密科技股份有限公司,台翰,28,782864190,上櫃,電子零組件業,電子零組件業,28
6,1565,精華光學股份有限公司,精華,22,504165160,上櫃,生技醫療業,生技醫療業,22
7,1569,濱川企業股份有限公司,濱川,25,1214474410,上櫃,電腦及週邊設備業,電腦及週邊設備業,25
8,1570,力肯實業股份有限公司,力肯,05,515008390,上櫃,電機機械,電機機械,05
9,1580,新麥企業股份有限公司,新麥,05,502302420,上櫃,電機機械,電機機械,05


In [ ]:
start_month='2023-01-01'  #起始月份
want_update_Stock=True
want_update_Revenue=False
file_path = 'D:\Project\Jupyter\Stock\Main\DailyResultLast.xlsx'   
#_baseInfo[_baseInfo['Type']!='上市']
for code in _baseInfo.dropna()['公司代號']:#.iloc[::-1]:
#for code in DDD:
    stock_number=str(code)
    if (want_update_Stock):        
        type_='Stock_His'
        strMonth = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-01')
        delete_DB_stock_by_Month_Stock_number(strMonth,stock_number,type_)
        if(datetime.now()<datetime.strptime('2024-11-10', '%Y-%m-%d')):
            strMonth = (datetime.now() - timedelta(days=39)).strftime('%Y-%m-01')
            delete_DB_stock_by_Month_Stock_number(strMonth,stock_number,type_)
    try:
        if(want_update_Revenue):
            MainStock(stock_number,start_month,want_update_Revenue)
        else:
            MainStock(stock_number,start_month)
    except Exception as error:
        print("An exception occurred:", error)
        print('Action Fail'+'\n')
        
    print(stock_number,'=====================>Data Ready')
   